In [1]:
import pandas as pd
import numpy as np

In [2]:
### Lectura De archivo
df = pd.read_csv("archivos/Super_Store_data.csv")

In [4]:
df.dtypes

Row ID             int64
Order ID          object
Order Date        object
Ship Date         object
Ship Mode         object
Customer ID       object
Customer Name     object
Segment           object
Country           object
City              object
State             object
Postal Code        int64
Region            object
Product ID        object
Category          object
Sub-Category      object
Product Name      object
Sales            float64
Quantity           int64
Discount         float64
Profit           float64
Coordinates       object
dtype: object

### tipos de variable 
- Enteros (int) 
- Decimal (float)
- Texto (objetct,str)
- Fecha (datetime)
- Categorico (category)

In [ ]:
### Modificar las fechas
df['Order Date'] = pd.to_datetime(df['Order Date'])
df = pd.read_csv("archivos/Super_Store_data.csv",
                 parse_dates=['Order Date','Ship Date'])

In [ ]:
df['Region']=df['Region'].astype('category')
### Lo mismo para 'Sub-Category'

In [ ]:
df = df[df["Coordinates"].str.contains(",")]

In [ ]:
df["Coordinates"] =df[""].str.replace("(","",regex=False).str.replace(")","",regex=False)
df[['lat', 'long']]=df[""].str.split(",",expand=True)
df.drop(columns=[""],inplace=True)

In [ ]:
df["price"]= df["Sales"]/df["Quantity"]

In [ ]:
### Metodologia basica para imputar los vacios
df['sales'].fillna(0)
#df['Sales'].fillna(df['Sales'].mean())

In [ ]:
prom = df.loc[df["Sales"].isnull()].groupby(["Product ID"])["price"].agg(["mean"]).reset_index()
sales_w=df.loc[df["Sales"].isnull()]
df=df.loc[~df["Sales"].isnull()]
sales_w=sales_w.merge(prom, how="left",on="Product ID")
sales_w["Sales"]=sales_w["Quantity"]*sales_w["mean"]
sales_w=sales_w.loc[:,df.columns]
df = pd.concat([df,sales_w])

In [ ]:
# retirar profits negativos
df = df.loc[df["Profit"]>0]

In [ ]:
df = df.loc[~(df["Profit"]>df["Sales"])]

In [ ]:
#valores atipicos
def valor_atipico(df,x):
    q3=df[x].quantile(0.75)
    q1=df[x].quantile(0.25)
    iqr=q3-q1
    print ("Cantidad de valores atipicos de la columna "+x+": ",df.loc[(df[x]>q3+1.5*iqr)|(df[x]<q1-1.5*iqr)].shape[0])
    df_nuevo = df.loc[(df[x]<=q3+1.5*iqr)&(df[x]>=q1-1.5*iqr)]
    return df_nuevo

In [ ]:
df_nuevo= valor_atipico(df,"Sales")

In [ ]:
df_nuevo["lat"] = df_nuevo["lat"].astype(float)
df_nuevo["long"] = df_nuevo["long"].astype(float)
df_nuevo.drop(columns=["price"],inplace=True)


In [ ]:
df_nuevo.isnull().sum()

Row ID           0
Order ID         0
Order Date       0
Ship Date        0
Ship Mode        0
Customer ID      0
Customer Name    0
Segment          0
Country          0
City             0
State            0
Postal Code      0
Region           0
Product ID       0
Category         0
Sub-Category     0
Product Name     0
Sales            0
Quantity         0
Discount         0
Profit           0
Coordinates      0
dtype: int64

In [ ]:
df_nuevo.to_csv("archivos/clean_data.csv")

### Prueba adicional
Hay forma de ver si hay una novedad con las fechas de entrega y del pedido